In [1]:
# in a terminal run
# > USER_PATH=~/userdata/ make run-redis NS=train
# > USER_PATH=~/userdata/ make run-redis NS=test
# to allow access to the train and test namespaces

In [2]:
import os
import sys
import json
import pandas as pd
import numpy as np
from typing import Literal, TypedDict

In [3]:
sys.path.append("..")
os.environ["USER_PATH"] = "/home/krause/userdata/"
MODEL_OUTPUT_BASE = "/mnt/d/workspace/clotho/notebooks"
MODEL_OUTPUT_CP = os.path.join(MODEL_OUTPUT_BASE, "checkpoints")

In [4]:
from misc.redis import set_redis_slow_mode
from misc.util import highest_number
from misc.io import open_write
from model.datagenerator import create_train_test
from model.transformer_embed import (
    get_epoch_and_load,
    limit_epoch_data,
    limit_epoch_data,
    get_model_filename,
)
from system.namespace.store import get_namespace

2023-02-07 09:48:36.560539: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-07 09:48:37.071620: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-07 09:48:37.071675: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-07 09:48:37.071679: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [5]:
import torch

is_cuda = torch.cuda.is_available()
is_cuda

True

In [6]:
set_redis_slow_mode("never")
ns_test = get_namespace("test")
ns_train = get_namespace("train")
now = pd.Timestamp("2022-12-17", tz="UTC")
train_plan = [
    {
        "left": {"mode": "valid", "flip_pc": 1.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 10,
        "last_epoch": None,
        "weight": 50,
    },
    {
        "left": {"mode": "valid", "flip_pc": 1.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 20,
        "last_epoch": None,
        "weight": 50,
    },
    # {
    #     "left": {"mode": "random", "flip_pc": 0.0},
    #     "right": {"mode": "path", "flip_pc": 0.0},
    #     "min_text_length": None,
    #     "skip_weak": True,
    #     "skip_topics": True,
    #     "flip_lr": 0.5,
    #     "first_epoch": None,
    #     "last_epoch": None,
    #     "weight": 60,
    # },
    # {
    #     "left": None,
    #     "right": {"mode": "path", "flip_pc": 0.0},
    #     "min_text_length": None,
    #     "skip_weak": True,
    #     "skip_topics": True,
    #     "flip_lr": 0.5,
    #     "first_epoch": None,
    #     "last_epoch": None,
    #     "weight": 40,
    # },
    # {
    #     "left": {"mode": "random", "flip_pc": 0.0},
    #     "right": {"mode": "path", "flip_pc": 0.0},
    #     "min_text_length": None,
    #     "skip_weak": False,
    #     "skip_topics": True,
    #     "flip_lr": 0.5,
    #     "first_epoch": 5,
    #     "last_epoch": None,
    #     "weight": 60,
    # },
    # {
    #     "left": None,
    #     "right": {"mode": "path", "flip_pc": 0.0},
    #     "min_text_length": None,
    #     "skip_weak": True,
    #     "skip_topics": True,
    #     "flip_lr": 0.5,
    #     "first_epoch": 5,
    #     "last_epoch": None,
    #     "weight": 40,
    # },
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 40,
    },
    {
        "left": {"mode": "valid", "flip_pc": 1.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 15,
        "last_epoch": None,
        "weight": 50,
    }
]
eval_plan = [
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 40,
    },
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 40,
    },
]
ttgen = create_train_test(
    train_ns=ns_train,
    train_validation_ns=ns_train,
    test_ns=ns_test,
    test_validation_ns=ns_test,
    train_learning_plan=train_plan,
    train_val_learning_plan=eval_plan,
    test_learning_plan=eval_plan,
    test_val_learning_plan=eval_plan,
    batch_size=4 if is_cuda else 8,
    epoch_batches=5000 if is_cuda else 500,
    train_val_size=10000 if is_cuda else 1000,
    test_size=10000 if is_cuda else 1000,
    test_val_size=10000 if is_cuda else 1000,
    use_fast_gen_only=True,
    compute_batch_size=100 if is_cuda else 100,
    now=now)

In [7]:
import torch.nn as nn
from torch.optim import AdamW
from transformers import DistilBertTokenizer, DistilBertModel

In [8]:
device = torch.device("cuda") if is_cuda else torch.device("cpu")
device

device(type='cuda')

In [9]:
from model.transformer_embed import (
    EMBED_SIZE,
    TokenizedInput,
    Model,
    BaselineModel,
    EitherModel,
    TrainingHarness,
    get_tokenizer,
)


tokens = get_tokenizer()

In [10]:
from transformers import get_scheduler
# from tqdm.notebook import tqdm
from tqdm.auto import tqdm
import evaluate
import time


def create_model(version: int) -> EitherModel:
    if version < 0:
        return BaselineModel(version, ignore_pretrained_warning=True)
    return Model(version, ignore_pretrained_warning=True)


def compute(harness, df):
    plefts = tokens(df["parent_left"].tolist())
    clefts = tokens(df["child_left"].tolist())
    prights = tokens(df["parent_right"].tolist())
    crights = tokens(df["child_right"].tolist())
    labels = torch.tensor(
        [~df["correct_is_right"], df["correct_is_right"]],
        dtype=torch.float32).T.to(device)
   
    preds, loss = harness(
        left={"parent": plefts, "child": clefts},
        right={"parent": prights, "child": crights},
        labels=labels)
    # TODO add selective push losses
    return preds, loss


def run_training(num_epochs, version, force_restart):
    model = create_model(version)
    model.to(device)
    harness = TrainingHarness(model)
    harness.to(device)

    mprev, epoch_offset = get_epoch_and_load(
        harness,
        MODEL_OUTPUT_CP,
        ftype="harness",
        is_cuda=is_cuda,
        device=device,
        force_restart=force_restart)

    optimizer = AdamW(harness.parameters(), lr=5e-5)
    print(mprev, epoch_offset)
    
    num_epochs -= epoch_offset
    if num_epochs <= 0:
        print("already computed all epochs. nothing to do!")
        return model, harness, optimizer
    
    num_training_steps = num_epochs * ttgen.get_epoch_train_size()
    warmup = 10000 if is_cuda else 10
    lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=warmup,
        num_training_steps=num_training_steps - warmup)
    ttgen.set_epoch(epoch_offset)
    
    log_csv = get_model_filename(
        harness,
        MODEL_OUTPUT_BASE,
        is_cuda=is_cuda,
        ftype="val_log",
        epoch=None,
        ext=".csv")
    columns = [
        "epoch",
        "train_acc",
        "train_loss",
        "train_val_acc",
        "train_val_loss",
        "test_acc",
        "test_loss",
        "time",
        "version",
        "fname",
    ]
    if not os.path.exists(log_csv):
        pd.DataFrame([], columns=columns).to_csv(
            log_csv, header=True, mode="w", columns=columns)

    for _ in range(num_epochs):
        epoch = ttgen.get_epoch()
        print(f"epoch {epoch} version: {harness.get_version()}")
        real_time = time.monotonic()

        model.train()
        harness.train()
        model.set_epoch(epoch)
        metric_train = evaluate.load("accuracy")
        train_loss = []
        first = True
        with tqdm(desc="train", total=ttgen.get_epoch_train_size()) as progress_bar:
            for train_df in ttgen.train_dfs():
                preds, loss = compute(harness, train_df)
                train_loss.append(loss.item())
                loss.backward()

                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(train_df.shape[0])

                predictions = torch.argmax(preds, dim=-1)
                metric_train.add_batch(
                    predictions=predictions,
                    references=train_df["correct_is_right"].astype(int))
                if first:
                    # display(train_df)
                    first = False

        model_fname = get_model_filename(
            harness,
            MODEL_OUTPUT_CP,
            is_cuda=is_cuda,
            ftype="harness",
            epoch=epoch)
        torch.save(harness.state_dict(), model_fname)

        model.eval()
        harness.eval()
        with torch.no_grad():
            metric_val_train = evaluate.load("accuracy")
            train_val_loss = []
            with tqdm(desc="train val", total=ttgen.get_epoch_train_validation_size()) as progress_bar:
                for train_validation_df in ttgen.train_validation_dfs():
                    preds, loss = compute(harness, train_validation_df)
                    train_val_loss.append(loss.item())
                    predictions = torch.argmax(preds, dim=-1)
                    metric_val_train.add_batch(
                        predictions=predictions,
                        references=train_validation_df["correct_is_right"].astype(int))
                    progress_bar.update(train_validation_df.shape[0])

            metric_test = evaluate.load("accuracy")
            test_loss = []
            with tqdm(desc="test", total=ttgen.get_epoch_test_size()) as progress_bar:
                for test_df in ttgen.test_dfs():
                    preds, loss = compute(harness, test_df)
                    test_loss.append(loss.item())
                    predictions = torch.argmax(preds, dim=-1)
                    metric_test.add_batch(
                        predictions=predictions,
                        references=test_df["correct_is_right"].astype(int))
                    progress_bar.update(test_df.shape[0])
            stats = {
                "epoch": int(epoch),
                "train_acc": float(metric_train.compute()['accuracy']),
                "train_loss": float(np.mean(train_loss)),
                "train_val_acc": float(metric_val_train.compute()['accuracy']),
                "train_val_loss": float(np.mean(train_val_loss)),
                "test_acc": float(metric_test.compute()['accuracy']),
                "test_loss": float(np.mean(test_loss)),
                "time": 0.0,
                "version": harness.get_version(),
                "fname": model_fname,
            }

        print(f"train: {stats['train_acc']} loss: {stats['train_loss']}")
        print(f"train val: {stats['train_val_acc']} loss: {stats['train_val_loss']}")
        print(f"test: {stats['test_acc']} loss: {stats['test_loss']}")
        ttgen.advance_epoch()
        stats["time"] = float((time.monotonic() - real_time) / 60.0)
        print(f"epoch time: {stats['time']:.2f}min")
        stats_fn = get_model_filename(
            harness,
            MODEL_OUTPUT_CP,
            is_cuda=is_cuda,
            ftype="stats",
            epoch=epoch,
            ext=".json")
        with open_write(stats_fn, text=True) as fout:
            print(json.dumps(stats, indent=2, sort_keys=True), file=fout)
        stats_df = pd.DataFrame(
            {key: [val] for key, val in stats.items()},
            columns=columns)
        stats_df.to_csv(
            log_csv, header=False, mode="a")
            
        limit_epoch_data(
            harness,
            MODEL_OUTPUT_CP,
            is_cuda=is_cuda,
            ftype="stats",
            ext=".json",
            count=5)
    return model, harness, optimizer

In [11]:
def save_model(model, harness, optimizer):
    torch.save(model.state_dict(), get_model_filename(
        harness,
        MODEL_OUTPUT_BASE,
        is_cuda=is_cuda,
        ftype="model",
        epoch=None))
    torch.save(harness.state_dict(), get_model_filename(
        harness,
        MODEL_OUTPUT_BASE,
        is_cuda=is_cuda,
        ftype="harness",
        epoch=None))
    torch.save(optimizer.state_dict(), get_model_filename(
        harness,
        MODEL_OUTPUT_BASE,
        is_cuda=is_cuda,
        ftype="optimizer",
        epoch=None))

In [12]:
def validation(model, harness):
    ttgen.reset()
    model.eval()
    harness.eval()
    dfs = []
    with torch.no_grad():
        metric_val_test = evaluate.load("accuracy")
        test_val_loss = []
        with tqdm(desc="test val", total=ttgen.get_epoch_test_validation_size()) as progress_bar:
            for test_val_df in ttgen.test_validation_dfs():
                preds, loss = compute(harness, test_val_df)
                test_val_loss.append(loss.item())
                predictions = torch.argmax(preds, dim=-1)
                metric_val_test.add_batch(
                    predictions=predictions,
                    references=test_val_df["correct_is_right"].astype(int))
                cur_df = test_val_df.copy()
                cur_df["logit_left"] = preds[:, 0].cpu()
                cur_df["logit_right"] = preds[:, 1].cpu()
                cur_df["preds"] = predictions.cpu()
                cur_df["truth"] = test_val_df["correct_is_right"].astype(int)
                dfs.append(cur_df)
                progress_bar.update(test_val_df.shape[0])
    print(f"test val: {metric_val_test.compute()} loss: {np.mean(test_val_loss)}")
    validation_df = pd.concat(dfs)
    validation_df.to_csv(get_model_filename(
        harness,
        MODEL_OUTPUT_BASE,
        is_cuda=is_cuda,
        ftype="validation",
        epoch=None,
        ext=".csv"))
    print("correct")
    display(validation_df[validation_df["preds"] == validation_df["truth"]].head())
    print("incorrect")
    display(validation_df[validation_df["preds"] != validation_df["truth"]].head())

In [13]:
def embeds(model):
    ttgen.reset()
    model.eval()
    with torch.no_grad():
        count = 0
        for test_val_df in ttgen.test_validation_dfs():
            plefts = tokens(test_val_df["parent_left"].tolist())
            clefts = tokens(test_val_df["child_left"].tolist())
            prights = tokens(test_val_df["parent_right"].tolist())
            crights = tokens(test_val_df["child_right"].tolist())
            display(model.get_child_embed(
                clefts["input_ids"],
                clefts["attention_mask"]).cpu().numpy())
            display(model.get_child_embed(
                crights["input_ids"],
                crights["attention_mask"]).cpu().numpy())
            count += 1
            if count >= 5:
                break

In [14]:
def full_run(*, num_epochs, version, force_restart):
    model, harness, optimizer = run_training(
        num_epochs, version, force_restart)
    save_model(model, harness, optimizer)
    validation(model, harness)
    embeds(model)

In [15]:
# for version in range(8):
#     full_run(num_epochs=30, version=version, force_restart=False)

In [16]:
full_run(num_epochs=10, version=-1, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


None 0
epoch 0 version: -1


train:   0%|          | 0/20000 [00:00<?, ?it/s]

built strong_links_cache in 58.8546s (275808 links)


train val:   0%|          | 0/10000 [00:00<?, ?it/s]

...

array([[-0.31996   , -0.18189602,  0.04552735, ..., -0.02894457,
        -0.51777965,  0.10447377],
       [-0.22432768, -0.26319695, -0.00541963, ..., -0.23549968,
        -0.44711626,  0.14477317],
       [-0.42644092, -0.31610385, -0.00110136, ..., -0.19524191,
        -0.4225089 ,  0.16476563],
       [-0.17943653, -0.04492769, -0.06243049, ..., -0.03048744,
        -0.5084228 ,  0.09863101]], dtype=float32)

array([[-0.28534377, -0.25085065,  0.05657804, ..., -0.09019367,
        -0.49878597,  0.07130849],
       [-0.28549457, -0.15648247,  0.03173396, ..., -0.13362385,
        -0.51019704,  0.17323522],
       [-0.3285185 , -0.13108797,  0.00783163, ..., -0.06438348,
        -0.5392172 ,  0.16921283],
       [-0.2580646 , -0.25254366, -0.00509375, ..., -0.22343965,
        -0.54704016, -0.12402717]], dtype=float32)

array([[-0.23507394, -0.2149986 ,  0.15219322, ..., -0.0545599 ,
        -0.53707856,  0.19537094],
       [-0.3740637 , -0.21756266,  0.08900664, ..., -0.04310518,
        -0.4557495 ,  0.14327519],
       [-0.24479079, -0.15962875,  0.00336285, ...,  0.02736009,
        -0.5058877 ,  0.11675741],
       [-0.29903898, -0.13651219,  0.00763706, ..., -0.07129624,
        -0.46715093,  0.23860513]], dtype=float32)

array([[-0.24545267, -0.04849231, -0.02618548, ..., -0.04108666,
        -0.45333636,  0.18225846],
       [-0.18380015, -0.09204808,  0.12611233, ..., -0.1137217 ,
        -0.5861045 ,  0.15950808],
       [-0.3506324 , -0.3963967 ,  0.03940408, ..., -0.24258454,
        -0.4424194 ,  0.33546925],
       [-0.1995926 , -0.19216421, -0.01602685, ..., -0.10625524,
        -0.505377  ,  0.12715074]], dtype=float32)

array([[-0.2836144 , -0.15677814, -0.09459592, ..., -0.06293986,
        -0.49806505,  0.03525713],
       [-0.1720153 , -0.04246559,  0.01070807, ..., -0.05250976,
        -0.44890565,  0.1713177 ],
       [-0.33019862, -0.19076915,  0.06658162, ..., -0.14227286,
        -0.38891786,  0.30292824],
       [-0.35658443, -0.40284097,  0.08302368, ..., -0.10343052,
        -0.5820363 ,  0.24354072]], dtype=float32)

In [17]:
full_run(num_epochs=10, version=7, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- T

None 0
epoch 0 version: 7


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

array([[ 0.15317209, -0.01637083,  0.3324464 , ..., -0.26272395,
        -0.4094483 , -0.39147052],
       [ 0.15112469, -0.01707165,  0.33510372, ..., -0.26316535,
        -0.40688244, -0.39087895],
       [ 0.15414964, -0.01612075,  0.3308985 , ..., -0.26227653,
        -0.41103074, -0.39128587],
       [ 0.15379551, -0.01631948,  0.33170503, ..., -0.26253867,
        -0.4101446 , -0.3915898 ]], dtype=float32)

array([[ 0.15259278, -0.01645915,  0.33284116, ..., -0.262647  ,
        -0.40916398, -0.39093488],
       [ 0.15410806, -0.01619131,  0.33105955, ..., -0.2622943 ,
        -0.41087475, -0.3913051 ],
       [ 0.15378176, -0.01658783,  0.3319818 , ..., -0.26271415,
        -0.40984014, -0.39181912],
       [ 0.15387282, -0.01604198,  0.33126932, ..., -0.2623554 ,
        -0.41060454, -0.3912958 ]], dtype=float32)

array([[ 0.15358977, -0.01625163,  0.33167383, ..., -0.26248142,
        -0.41021925, -0.39125365],
       [ 0.1538627 , -0.01621407,  0.331552  , ..., -0.2625309 ,
        -0.4102168 , -0.39165995],
       [ 0.15382624, -0.01627597,  0.33154655, ..., -0.26249218,
        -0.41029543, -0.39150324],
       [ 0.15382366, -0.01648651,  0.3317377 , ..., -0.26261643,
        -0.41005772, -0.3917225 ]], dtype=float32)

array([[ 0.15360354, -0.01634669,  0.331905  , ..., -0.2626833 ,
        -0.40991956, -0.39157563],
       [ 0.15343067, -0.01616175,  0.3318389 , ..., -0.26247603,
        -0.41006944, -0.39119908],
       [ 0.15363127, -0.01622319,  0.33185542, ..., -0.26258898,
        -0.41001844, -0.391536  ],
       [ 0.1542004 , -0.01581695,  0.33049482, ..., -0.26213753,
        -0.41141093, -0.39106625]], dtype=float32)

array([[ 0.15199007, -0.01673173,  0.333813  , ..., -0.26293018,
        -0.40814906, -0.39097825],
       [ 0.1535879 , -0.01627171,  0.33200118, ..., -0.2626154 ,
        -0.4098485 , -0.39165238],
       [ 0.15386835, -0.01609899,  0.33130145, ..., -0.26239312,
        -0.41058218, -0.39131835],
       [ 0.15377785, -0.01626401,  0.3316562 , ..., -0.26250106,
        -0.41019544, -0.39151612]], dtype=float32)

In [18]:
full_run(num_epochs=10, version=0, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- T

None 0
epoch 0 version: 0


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

array([[-0.4606564 ,  0.08024991,  0.0165747 , ...,  0.5906613 ,
        -0.19296117, -0.04994956],
       [-0.46711433,  0.07688809,  0.00388761, ...,  0.6029753 ,
        -0.19947438, -0.05427572],
       [-0.45682853,  0.0821236 ,  0.02410733, ...,  0.58319867,
        -0.18914217, -0.04719081],
       [-0.46270558,  0.07897757,  0.01270098, ...,  0.59443027,
        -0.1945493 , -0.05174243]], dtype=float32)

array([[-0.4616736 ,  0.07975656,  0.01457711, ...,  0.5925977 ,
        -0.19401273, -0.05059686],
       [-0.45543495,  0.08277738,  0.02685146, ...,  0.58042043,
        -0.18778118, -0.04611072],
       [-0.4599418 ,  0.08048163,  0.01792709, ...,  0.5894423 ,
        -0.19233714, -0.04966655],
       [-0.4590897 ,  0.08102452,  0.01964624, ...,  0.58763206,
        -0.19136946, -0.04886032]], dtype=float32)

array([[-0.4596514 ,  0.08075567,  0.018542  , ...,  0.5887022 ,
        -0.1919242 , -0.04923059],
       [-0.4634743 ,  0.0785196 ,  0.01219696, ...,  0.59481853,
        -0.19419654, -0.05189609],
       [-0.460584  ,  0.08031858,  0.01670034, ...,  0.5904858 ,
        -0.19286123, -0.049862  ],
       [-0.4616337 ,  0.0795188 ,  0.01459071, ...,  0.5926226 ,
        -0.19360392, -0.05114024]], dtype=float32)

array([[-0.45932078,  0.0809054 ,  0.01918497, ...,  0.5881377 ,
        -0.19163613, -0.04904153],
       [-0.46132892,  0.07981806,  0.01542836, ...,  0.5918567 ,
        -0.19340846, -0.05046479],
       [-0.4607174 ,  0.08023208,  0.01646917, ...,  0.5907444 ,
        -0.19299553, -0.04994405],
       [-0.4556565 ,  0.08264702,  0.02641972, ...,  0.5809186 ,
        -0.1879809 , -0.04639998]], dtype=float32)

array([[-0.46533385,  0.07775874,  0.00788106, ...,  0.5988517 ,
        -0.19684808, -0.05323636],
       [-0.45999107,  0.08060847,  0.01790004, ...,  0.589321  ,
        -0.19223897, -0.04943401],
       [-0.45952138,  0.08083861,  0.01881001, ...,  0.5884512 ,
        -0.19178158, -0.04914878],
       [-0.46184498,  0.07928681,  0.01474312, ...,  0.5924585 ,
        -0.19344038, -0.05107198]], dtype=float32)

In [19]:
full_run(num_epochs=10, version=5, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- T

None 0
epoch 0 version: 5


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

array([[-0.4446466 ,  0.16472916,  1.7887194 , ...,  0.21003033,
         0.7549762 ,  0.26538116],
       [-0.44466233,  0.16473214,  1.78871   , ...,  0.21003981,
         0.7550081 ,  0.26537806],
       [-0.44466135,  0.16479735,  1.7886525 , ...,  0.21007645,
         0.7550349 ,  0.2653579 ],
       [-0.44465277,  0.16479829,  1.7886556 , ...,  0.21007927,
         0.75502414,  0.2653583 ]], dtype=float32)

array([[-0.44464773,  0.16473775,  1.788715  , ...,  0.21003178,
         0.75497925,  0.26538152],
       [-0.44465244,  0.16479717,  1.7886583 , ...,  0.21008062,
         0.75502247,  0.26535922],
       [-0.44465217,  0.16479768,  1.7886572 , ...,  0.21008056,
         0.7550222 ,  0.26535848],
       [-0.44464484,  0.16473027,  1.7887256 , ...,  0.21002483,
         0.75496656,  0.26538375]], dtype=float32)

array([[-0.44466367,  0.1647391 ,  1.7887036 , ...,  0.21004637,
         0.75501627,  0.26537466],
       [-0.4446453 ,  0.16472583,  1.7887255 , ...,  0.21002299,
         0.754956  ,  0.265379  ],
       [-0.4446523 ,  0.16473037,  1.7887186 , ...,  0.21003224,
         0.75498337,  0.26538077],
       [-0.44465107,  0.1647973 ,  1.7886585 , ...,  0.21008077,
         0.75502014,  0.26535875]], dtype=float32)

array([[-0.4446519 ,  0.16479881,  1.7886555 , ...,  0.21007937,
         0.7550225 ,  0.26535836],
       [-0.4446618 ,  0.16475886,  1.7886858 , ...,  0.21005493,
         0.7550226 ,  0.26537266],
       [-0.44466838,  0.16476107,  1.7886758 , ...,  0.21005951,
         0.75503117,  0.26537117],
       [-0.4446548 ,  0.16480014,  1.7886482 , ...,  0.21007669,
         0.7550296 ,  0.26535547]], dtype=float32)

array([[-0.44466224,  0.16478032,  1.7886703 , ...,  0.21006854,
         0.75503945,  0.26536435],
       [-0.44466528,  0.16473693,  1.7887007 , ...,  0.21004643,
         0.7550191 ,  0.2653742 ],
       [-0.44464275,  0.16473179,  1.7887274 , ...,  0.21002187,
         0.7549559 ,  0.26538557],
       [-0.4446629 ,  0.16474798,  1.7886933 , ...,  0.21004863,
         0.75501585,  0.2653735 ]], dtype=float32)

In [16]:
full_run(num_epochs=10, version=-3, force_restart=False)

None 0
epoch 0 version: -3


train:   0%|          | 0/20000 [00:00<?, ?it/s]

built strong_links_cache in 70.6575s (275808 links)


train val:   0%|          | 0/10000 [00:00<?, ?it/s]

built all_links_cache in 76.2563s (605713 links)


...

array([[-0.01216118, -0.01027532,  0.05986409, ...,  0.00814815,
         0.0119706 ,  0.0627512 ],
       [ 0.15700738, -0.0628274 ,  0.03205046, ..., -0.04882665,
         0.03913112, -0.01106808],
       [-0.04884599,  0.03536889,  0.01952939, ..., -0.01736142,
        -0.01588253,  0.07578498],
       [-0.0386449 ,  0.08544032, -0.02326893, ..., -0.00108779,
        -0.05551765,  0.03536721]], dtype=float32)

array([[-0.04273321,  0.01149174,  0.05313378, ...,  0.00142086,
        -0.00078021,  0.04105023],
       [-0.03048875,  0.0349448 ,  0.01686766, ...,  0.00365246,
        -0.02248074,  0.06952373],
       [-0.07949167,  0.01704806,  0.00135581, ...,  0.00447649,
        -0.07217579,  0.02655156],
       [-0.07770352, -0.09272111,  0.04150224, ..., -0.00820773,
        -0.12882787, -0.13225487]], dtype=float32)

array([[-0.02874596, -0.04377675,  0.02673315, ..., -0.02858031,
        -0.09688856,  0.01075301],
       [ 0.04059798, -0.08266897,  0.038612  , ..., -0.01230833,
         0.05540211,  0.03382163],
       [ 0.07226831, -0.03781992,  0.06010635, ..., -0.04824931,
         0.02993268,  0.05259719],
       [-0.07388042,  0.06284337, -0.02998865, ...,  0.00921352,
         0.00510864,  0.0192851 ]], dtype=float32)

array([[-0.02623415,  0.07114965,  0.04847452, ...,  0.03733629,
        -0.04904086,  0.02071522],
       [-0.07902232, -0.00171336, -0.03483646, ..., -0.00937016,
        -0.11148877, -0.07994886],
       [-0.00438465,  0.00019584,  0.04109041, ..., -0.01610458,
         0.02371685,  0.06993052],
       [-0.15049002, -0.02222895, -0.02070254, ...,  0.01974531,
        -0.06894305,  0.01096964]], dtype=float32)

array([[ 0.04715401, -0.03544792,  0.02919866, ..., -0.02807448,
        -0.02867752,  0.04718057],
       [-0.00957002, -0.1633473 , -0.02208889, ..., -0.00587601,
        -0.06482904,  0.01275759],
       [-0.07073933,  0.08292375, -0.01460287, ...,  0.0041601 ,
        -0.04598795,  0.00825759],
       [-0.04071502,  0.00965045, -0.00085335, ..., -0.03802673,
        -0.10826242, -0.01524383]], dtype=float32)

In [20]:
full_run(num_epochs=30, version=-1, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


('harness_v-1_lg_9.pkl', 9) 10
epoch 10 version: -1


train:   0%|          | 0/20000 [00:00<?, ?it/s]

built all_links_cache in 68.8344s (605713 links)


train val:   0%|          | 0/10000 [00:00<?, ?it/s]

...

array([[-0.21961202, -0.04409703,  0.05316635, ...,  0.38636613,
        -0.11702938, -0.00676587],
       [-0.20398249, -0.07169177, -0.00202861, ...,  0.37212148,
        -0.16131754,  0.02692908],
       [-0.28699008, -0.07522519,  0.05004616, ...,  0.3264634 ,
        -0.08981069,  0.0399402 ],
       [-0.25826797, -0.07547107,  0.06327439, ...,  0.32132956,
        -0.12475533,  0.034598  ]], dtype=float32)

array([[-0.22160861, -0.04976817,  0.03992751, ...,  0.3760917 ,
        -0.12957464,  0.00533186],
       [-0.28045282, -0.08875145,  0.05451649, ...,  0.32292292,
        -0.08700256,  0.0470499 ],
       [-0.26689667, -0.06571285,  0.07573073, ...,  0.32195577,
        -0.10957669,  0.03330638],
       [-0.15940319, -0.10788478,  0.00121373, ...,  0.37272626,
        -0.17257047,  0.01053531]], dtype=float32)

array([[-0.1992735 , -0.03889766,  0.07838546, ...,  0.38821036,
        -0.1306535 ,  0.01420154],
       [-0.19906367, -0.03751208,  0.06115982, ...,  0.40918177,
        -0.0956584 , -0.02325248],
       [-0.21448606, -0.041462  ,  0.05891516, ...,  0.3911989 ,
        -0.10137113,  0.00237555],
       [-0.26895744, -0.07274017,  0.05818442, ...,  0.3270503 ,
        -0.11442769,  0.02584561]], dtype=float32)

array([[-0.25777555, -0.10431144,  0.0057184 , ...,  0.33074743,
        -0.13257839,  0.05520876],
       [-0.20931312, -0.04331321,  0.08796676, ...,  0.33835173,
        -0.12737481,  0.01743154],
       [-0.22849943, -0.09178904, -0.00588514, ...,  0.34379232,
        -0.10203103,  0.06130239],
       [-0.23603255, -0.05493734,  0.06183036, ...,  0.38459352,
        -0.07469484,  0.0087808 ]], dtype=float32)

array([[-2.24060282e-01, -8.54282305e-02,  5.69192460e-03, ...,
         3.67946833e-01, -1.16841555e-01,  5.04465178e-02],
       [-1.96839437e-01, -2.44045239e-02,  3.05816494e-02, ...,
         3.87275487e-01, -1.14892118e-01,  3.29019618e-04],
       [-2.24905625e-01, -4.28817756e-02,  5.75421154e-02, ...,
         3.55464488e-01, -1.43760338e-01, -4.51324834e-03],
       [-2.15533048e-01, -9.41237807e-02,  4.65311445e-02, ...,
         3.64747316e-01, -1.20042078e-01,  3.17580700e-02]], dtype=float32)

In [21]:
full_run(num_epochs=30, version=7, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- T

('harness_v7_lg_9.pkl', 9) 10
epoch 10 version: 7


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

array([[-0.07935055,  0.21158572,  0.01400581, ...,  0.29037917,
         0.10904216, -0.01462834],
       [-0.08490666,  0.21069957,  0.01225568, ...,  0.28471687,
         0.11371367, -0.01728587],
       [-0.07504256,  0.20838307,  0.01389816, ...,  0.2918318 ,
         0.10900675, -0.01474865],
       [-0.07762751,  0.21225423,  0.01452948, ...,  0.29232016,
         0.10716264, -0.01379664]], dtype=float32)

array([[-0.07795603,  0.21209046,  0.01443061, ...,  0.29193237,
         0.10756481, -0.01395315],
       [-0.07504303,  0.20838282,  0.01389799, ...,  0.29183152,
         0.10900712, -0.01474884],
       [-0.07727753,  0.21240996,  0.01462973, ...,  0.29271808,
         0.10675905, -0.01362717],
       [-0.07869792,  0.21175092,  0.01421007, ...,  0.2910747 ,
         0.10841942, -0.01431812]], dtype=float32)

array([[-0.07504371,  0.20838222,  0.01389779, ...,  0.29183102,
         0.10900774, -0.01474921],
       [-0.08224577,  0.2109459 ,  0.01308403, ...,  0.28732508,
         0.11168354, -0.01601104],
       [-0.07802614,  0.21219367,  0.01441499, ...,  0.29192385,
         0.1075049 , -0.0139851 ],
       [-0.07504339,  0.2083825 ,  0.01389794, ...,  0.29183125,
         0.10900749, -0.01474909]], dtype=float32)

array([[-0.07504279,  0.20838293,  0.01389811, ...,  0.2918316 ,
         0.10900691, -0.01474874],
       [-0.07883827,  0.21171436,  0.01416582, ...,  0.2909256 ,
         0.10855535, -0.01438385],
       [-0.07883434,  0.21171962,  0.01416536, ...,  0.2909298 ,
         0.10855313, -0.01437866],
       [-0.07504065,  0.20838454,  0.01389871, ...,  0.2918331 ,
         0.10900527, -0.01474755]], dtype=float32)

array([[-0.07778417,  0.21223882,  0.0144852 , ...,  0.29216912,
         0.10728837, -0.01387072],
       [-0.07867824,  0.21175914,  0.01421465, ...,  0.29109645,
         0.10840179, -0.01430578],
       [-0.07687777,  0.2124221 ,  0.01473345, ...,  0.29308596,
         0.10646579, -0.01343586],
       [-0.07787353,  0.2120403 ,  0.01445652, ...,  0.29198626,
         0.1075583 , -0.01391599]], dtype=float32)

In [22]:
full_run(num_epochs=30, version=0, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- T

('harness_v0_lg_9.pkl', 9) 10
epoch 10 version: 0


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

array([[-0.19641358,  0.1638949 , -0.05576622, ...,  0.61611634,
        -0.23020928,  0.15588959],
       [-0.20421572,  0.1707516 , -0.05883898, ...,  0.61781377,
        -0.23634212,  0.1699594 ],
       [-0.17337953,  0.1494838 , -0.04618713, ...,  0.61150223,
        -0.21529917,  0.12137192],
       [-0.18240395,  0.15514705, -0.04975638, ...,  0.61310023,
        -0.22114293,  0.13463421]], dtype=float32)

array([[-0.19246943,  0.16124313, -0.05412439, ...,  0.6152364 ,
        -0.22763155,  0.14969908],
       [-0.16975932,  0.14719453, -0.04480873, ...,  0.6109251 ,
        -0.21295175,  0.1160465 ],
       [-0.17175756,  0.14846031, -0.04556479, ...,  0.6112383 ,
        -0.21424747,  0.11898696],
       [-0.19060473,  0.16024835, -0.05350209, ...,  0.6148716 ,
        -0.22651757,  0.1469806 ]], dtype=float32)

array([[-0.19213846,  0.16119663, -0.05402029, ...,  0.61519367,
        -0.22748919,  0.1493094 ],
       [-0.19006215,  0.15996988, -0.05321482, ...,  0.6147754 ,
        -0.22626182,  0.14633766],
       [-0.19292988,  0.16155486, -0.05433647, ...,  0.6152889 ,
        -0.22790429,  0.15026657],
       [-0.1711714 ,  0.14808954, -0.04534175, ...,  0.61114496,
        -0.21386725,  0.1181245 ]], dtype=float32)

array([[-0.17171295,  0.14843212, -0.04554779, ...,  0.6112311 ,
        -0.2142186 ,  0.11892121],
       [-0.1885171 ,  0.15891275, -0.05228226, ...,  0.61431587,
        -0.22506978,  0.14367312],
       [-0.18433131,  0.15623218, -0.0509072 , ...,  0.6135764 ,
        -0.22249167,  0.13760167],
       [-0.17189452,  0.14854684, -0.04561708, ...,  0.61126024,
        -0.21433637,  0.11918832]], dtype=float32)

array([[-0.20390502,  0.16950344, -0.05898563, ...,  0.6178173 ,
        -0.23567957,  0.16822062],
       [-0.19555695,  0.1632354 , -0.05540396, ...,  0.6159294 ,
        -0.22963817,  0.15448786],
       [-0.18899806,  0.15928404, -0.05287395, ...,  0.6145487 ,
        -0.22555712,  0.1446712 ],
       [-0.18487792,  0.15655766, -0.05111208, ...,  0.61368376,
        -0.22283004,  0.13839918]], dtype=float32)

In [23]:
full_run(num_epochs=30, version=5, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- T

('harness_v5_lg_9.pkl', 9) 10
epoch 10 version: 5


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

array([[-0.53099066,  0.2778602 ,  0.81765246, ...,  0.15044427,
         0.4836439 ,  0.1925388 ],
       [-0.53299606,  0.2784095 ,  0.8046808 , ...,  0.15044232,
         0.4756812 ,  0.19382966],
       [-0.52792585,  0.27681896,  0.8415682 , ...,  0.15136534,
         0.49418715,  0.18922864],
       [-0.52944684,  0.27740088,  0.82869935, ...,  0.15071385,
         0.48905936,  0.19108708]], dtype=float32)

array([[-0.5299877 ,  0.2775706 ,  0.8246533 , ...,  0.15058358,
         0.4872043 ,  0.19164243],
       [-0.52875656,  0.27716064,  0.83423257, ...,  0.15095101,
         0.49139097,  0.19029994],
       [-0.5286148 ,  0.2771071 ,  0.8354279 , ...,  0.15101083,
         0.49186724,  0.19012685],
       [-0.53033984,  0.27767515,  0.82213056, ...,  0.15052097,
         0.48597524,  0.19197649]], dtype=float32)

array([[-0.52961093,  0.27745375,  0.8274462 , ...,  0.15066965,
         0.4884992 ,  0.19126134],
       [-0.53052074,  0.27772737,  0.8208641 , ...,  0.1504948 ,
         0.48533607,  0.19214003],
       [-0.532915  ,  0.27838692,  0.80519706, ...,  0.15043712,
         0.47603458,  0.19379076],
       [-0.5291836 ,  0.2773128 ,  0.8307564 , ...,  0.15079418,
         0.489952  ,  0.19079752]], dtype=float32)

array([[-0.5288774 ,  0.27720505,  0.83323056, ...,  0.15090299,
         0.4909844 ,  0.19044425],
       [-0.5296248 ,  0.27745813,  0.82734287, ...,  0.15066615,
         0.4884525 ,  0.19127552],
       [-0.5293188 ,  0.27735847,  0.8296927 , ...,  0.15075149,
         0.48949432,  0.19094773],
       [-0.5281911 ,  0.27693573,  0.8391378 , ...,  0.15121591,
         0.4932922 ,  0.18958572]], dtype=float32)

array([[-0.5316444 ,  0.2780401 ,  0.81333417, ...,  0.15040985,
         0.4812005 ,  0.19303328],
       [-0.5303452 ,  0.27767676,  0.8220925 , ...,  0.15052004,
         0.48595625,  0.19198139],
       [-0.52940685,  0.27738768,  0.8290084 , ...,  0.15072529,
         0.48919547,  0.19104385],
       [-0.52906525,  0.27727193,  0.8317017 , ...,  0.15083416,
         0.4903516 ,  0.19066307]], dtype=float32)

In [ ]:
full_run(num_epochs=30, version=-3, force_restart=False)

('harness_v-3_lg_9.pkl', 9) 10
epoch 10 version: -3


train:   0%|          | 0/20000 [00:00<?, ?it/s]

built all_links_cache in 74.4019s (605713 links)


train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

train: 0.80685 loss: 0.3486989788418019
train val: 0.8011 loss: 0.35198161785918164
test: 0.6712 loss: 0.6254208954334259
epoch time: 65.58min
removing /mnt/d/workspace/clotho/notebooks/checkpoints/stats_v-3_lg_22.json
removing /mnt/d/workspace/clotho/notebooks/checkpoints/harness_v-3_lg_22.pkl
best model: /mnt/d/workspace/clotho/notebooks/checkpoints/harness_v-3_lg_10.pkl
best train: 0.8325
best train val: 0.8604
best test: 0.7132
epoch 24 version: -3


train:   0%|          | 0/20000 [00:00<?, ?it/s]

In [24]:
full_run(num_epochs=60, version=-1, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


('harness_v-1_lg_29.pkl', 29) 30
epoch 30 version: -1


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

array([[-0.58793455,  0.34086514,  0.00699423, ...,  0.36443835,
         0.12329113,  0.41263485],
       [-0.58784735,  0.34189072,  0.0076062 , ...,  0.36373913,
         0.12405422,  0.41321236],
       [-0.58837193,  0.3380902 ,  0.00544527, ...,  0.36593166,
         0.12137273,  0.41103745],
       [-0.5883312 ,  0.3382985 ,  0.00556319, ...,  0.36583042,
         0.12151964,  0.41116017]], dtype=float32)

array([[-0.58795893,  0.340662  ,  0.00688368, ...,  0.3645541 ,
         0.12315752,  0.4125213 ],
       [-0.5883729 ,  0.33808625,  0.00544304, ...,  0.36593363,
         0.12136993,  0.4110352 ],
       [-0.58835995,  0.3381513 ,  0.00547981, ...,  0.3659022 ,
         0.12141586,  0.41107354],
       [-0.5878059 ,  0.34214476,  0.00768889, ...,  0.363657  ,
         0.12413554,  0.41334474]], dtype=float32)

array([[-0.5879802 ,  0.34050512,  0.00680256, ...,  0.3646392 ,
         0.12305925,  0.41243353],
       [-0.5878109 ,  0.34218523,  0.00773478, ...,  0.3635985 ,
         0.12419962,  0.41337088],
       [-0.58796793,  0.3405899 ,  0.00684264, ...,  0.36459923,
         0.12310588,  0.41248015],
       [-0.58837503,  0.3380751 ,  0.00543651, ...,  0.3659391 ,
         0.12136211,  0.41102877]], dtype=float32)

array([[-0.58835006,  0.33820117,  0.00550815, ...,  0.365878  ,
         0.12145105,  0.4111029 ],
       [-0.5883442 ,  0.3382308 ,  0.00552486, ...,  0.36586356,
         0.12147182,  0.41112024],
       [-0.5881028 ,  0.3396141 ,  0.0063012 , ...,  0.36514905,
         0.12243751,  0.41192508],
       [-0.58836937,  0.3381034 ,  0.0054525 , ...,  0.36592537,
         0.12138198,  0.41104525]], dtype=float32)

array([[-0.5883699 ,  0.33810148,  0.00545153, ...,  0.36592633,
         0.12138059,  0.41104406],
       [-0.5880477 ,  0.33999094,  0.00651343, ...,  0.3649398 ,
         0.12270054,  0.41214067],
       [-0.58797306,  0.3405475 ,  0.0068194 , ...,  0.36462444,
         0.12307748,  0.41245583],
       [-0.5881593 ,  0.33925718,  0.00610196, ...,  0.36534163,
         0.12219025,  0.4117193 ]], dtype=float32)

In [25]:
full_run(num_epochs=60, version=7, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- T

('harness_v7_lg_29.pkl', 29) 30
epoch 30 version: 7


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

array([[-0.11611613,  0.13499503, -0.03134627, ...,  0.3813403 ,
        -0.06548296, -0.06286929],
       [-0.11531435,  0.13648486, -0.02604626, ...,  0.3814594 ,
        -0.06325317, -0.06378379],
       [-0.1165837 ,  0.13412999, -0.03430605, ...,  0.38131976,
        -0.06679583, -0.06244906],
       [-0.11630034,  0.13474639, -0.03228001, ...,  0.38133085,
        -0.06588917, -0.06274219]], dtype=float32)

array([[-0.11657214,  0.13418776, -0.03413924, ...,  0.3813212 ,
        -0.06671771, -0.0624763 ],
       [-0.11664898,  0.13369483, -0.03551513, ...,  0.3813009 ,
        -0.06736799, -0.06224122],
       [-0.11647841,  0.13446042, -0.03329112, ...,  0.3813255 ,
        -0.06633121, -0.06260476],
       [-0.11649711,  0.13442045, -0.03342297, ...,  0.381325  ,
        -0.06639005, -0.06258588]], dtype=float32)

array([[-0.11657441,  0.1341773 , -0.03416977, ...,  0.38132098,
        -0.066732  , -0.06247136],
       [-0.11560237,  0.13580446, -0.02829164, ...,  0.38140658,
        -0.06409805, -0.0633503 ],
       [-0.1164401 ,  0.13453281, -0.03304578, ...,  0.38132665,
        -0.06622264, -0.06263904],
       [-0.1164284 ,  0.13455313, -0.0329755 , ...,  0.3813269 ,
        -0.06619169, -0.06264875]], dtype=float32)

array([[-0.1166673 ,  0.13358957, -0.03580701, ...,  0.38129398,
        -0.06750405, -0.06218981],
       [-0.11658295,  0.13413402, -0.03429453, ...,  0.38131985,
        -0.06679045, -0.06245095],
       [-0.11647995,  0.13445728, -0.03330158, ...,  0.3813255 ,
        -0.06633586, -0.06260327],
       [-0.11662871,  0.13382494, -0.03515565, ...,  0.38130808,
        -0.06719847, -0.062304  ]], dtype=float32)

array([[-0.11648158,  0.13445391, -0.03331285, ...,  0.38132545,
        -0.06634086, -0.06260166],
       [-0.1161534 ,  0.1349449 , -0.0315362 , ...,  0.381338  ,
        -0.0655662 , -0.06284288],
       [-0.11659382,  0.1340701 , -0.03447589, ...,  0.38131818,
        -0.0668759 , -0.06242072],
       [-0.11657241,  0.1341865 , -0.0341429 , ...,  0.3813211 ,
        -0.06671941, -0.06247572]], dtype=float32)

In [26]:
full_run(num_epochs=60, version=0, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- T

('harness_v0_lg_29.pkl', 29) 30
epoch 30 version: 0


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

array([[-0.1568382 , -0.01878631, -0.01939946, ...,  0.5396778 ,
        -0.15600371,  0.02231534],
       [-0.16102004, -0.0183615 , -0.02584347, ...,  0.5380642 ,
        -0.1612218 ,  0.02930584],
       [-0.14519006, -0.01996285, -0.00133567, ...,  0.544038  ,
        -0.14133257,  0.00292063],
       [-0.15234606, -0.01923918, -0.01245046, ...,  0.5413851 ,
        -0.15036829,  0.01482079]], dtype=float32)

array([[-0.14905149, -0.01957246, -0.00733951, ...,  0.5426166 ,
        -0.14621694,  0.00933504],
       [-0.1425048 , -0.02023575,  0.00284743, ...,  0.5450109 ,
        -0.13792355, -0.00153223],
       [-0.1460339 , -0.01987747, -0.00264887, ...,  0.54372936,
        -0.14240171,  0.00432102],
       [-0.1488291 , -0.01959497, -0.00699407, ...,  0.5426992 ,
        -0.14593616,  0.0089652 ]], dtype=float32)

array([[-0.1521188 , -0.01926212, -0.01209826, ...,  0.5414707 ,
        -0.15008242,  0.0144421 ],
       [-0.17347129, -0.01715314, -0.04481551, ...,  0.5330609 ,
        -0.17660055,  0.0501533 ],
       [-0.15293036, -0.01918023, -0.01335573, ...,  0.5411648 ,
        -0.15110302,  0.01579484],
       [-0.15074398, -0.01940111, -0.00996652, ...,  0.5419863 ,
        -0.14835158,  0.01215187]], dtype=float32)

array([[-1.4854120e-01, -1.9624081e-02, -6.5469453e-03, ...,
         5.4280579e-01, -1.4557265e-01,  8.4864181e-03],
       [-1.4928737e-01, -1.9548591e-02, -7.7058184e-03, ...,
         5.4252905e-01, -1.4651468e-01,  9.7274566e-03],
       [-1.5153971e-01, -1.9320633e-02, -1.1200608e-02, ...,
         5.4168814e-01, -1.4935368e-01,  1.3477181e-02],
       [-1.4351121e-01, -2.0133318e-02,  1.2789462e-03, ...,
         5.4464787e-01, -1.3920246e-01,  1.3612225e-04]], dtype=float32)

array([[-0.15247379, -0.01922628, -0.01264832, ...,  0.541337  ,
        -0.15052891,  0.01503368],
       [-0.1519192 , -0.01928228, -0.01178884, ...,  0.5415457 ,
        -0.14983125,  0.01410941],
       [-0.14891729, -0.01958603, -0.00713107, ...,  0.54266644,
        -0.1460475 ,  0.00911182],
       [-0.14787562, -0.01969134, -0.00551277, ...,  0.54305196,
        -0.14473173,  0.00738   ]], dtype=float32)

In [27]:
full_run(num_epochs=60, version=5, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- T

('harness_v5_lg_29.pkl', 29) 30
epoch 30 version: 5


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

array([[-0.1464946 ,  0.12814909,  0.4559949 , ...,  0.0968096 ,
         0.37121534,  0.12568246],
       [-0.14342034,  0.12928392,  0.4521368 , ...,  0.09739291,
         0.36986944,  0.12682748],
       [-0.15593646,  0.1246461 ,  0.46811324, ...,  0.09505542,
         0.3754719 ,  0.1223475 ],
       [-0.14808759,  0.1275621 ,  0.45801678, ...,  0.09650704,
         0.37192756,  0.12510137]], dtype=float32)

array([[-0.1515194 ,  0.12629533,  0.46240896, ...,  0.09585787,
         0.3734748 ,  0.12387601],
       [-0.15496354,  0.12501499,  0.46685165, ...,  0.09522177,
         0.3750268 ,  0.12267844],
       [-0.15281914,  0.12581526,  0.46408036, ...,  0.0956121 ,
         0.37405887,  0.12341929],
       [-0.15137997,  0.12634684,  0.46223003, ...,  0.09588432,
         0.3734121 ,  0.12392525]], dtype=float32)

array([[-0.15016834,  0.1267941 ,  0.4606765 , ...,  0.09611387,
         0.37286597,  0.12435492],
       [-0.14402017,  0.1290617 ,  0.45288578, ...,  0.09727987,
         0.37012893,  0.12660204],
       [-0.14810985,  0.12755392,  0.4580453 , ...,  0.09650288,
         0.37193757,  0.12509333],
       [-0.14732057,  0.12784484,  0.4570413 , ...,  0.09665255,
         0.37158343,  0.12538005]], dtype=float32)

array([[-0.15343213,  0.1255881 ,  0.46487087, ...,  0.09549767,
         0.37433425,  0.12320575],
       [-0.1514341 ,  0.12632681,  0.46229944, ...,  0.09587404,
         0.37343645,  0.12390612],
       [-0.15128866,  0.12638049,  0.46211287, ...,  0.09590162,
         0.37337103,  0.12395748],
       [-0.15775022,  0.12395966,  0.4704719 , ...,  0.09475022,
         0.37630534,  0.12173787]], dtype=float32)

array([[-0.14866216,  0.12735024,  0.45874962, ...,  0.09639839,
         0.37218627,  0.12489384],
       [-0.14981005,  0.12692638,  0.46021754, ...,  0.09618156,
         0.3727043 ,  0.12448258],
       [-0.15095091,  0.12650517,  0.46167958, ...,  0.09596562,
         0.3732189 ,  0.12407701],
       [-0.15042284,  0.12670013,  0.46100256, ...,  0.09606571,
         0.37298083,  0.1242644 ]], dtype=float32)

In [ ]:
full_run(num_epochs=60, version=-3, force_restart=False)

In [ ]:
full_run(num_epochs=90, version=-1, force_restart=False)

In [ ]:
full_run(num_epochs=90, version=7, force_restart=False)

In [ ]:
full_run(num_epochs=90, version=0, force_restart=False)

In [ ]:
full_run(num_epochs=90, version=5, force_restart=False)

In [ ]:
full_run(num_epochs=90, version=-3, force_restart=False)

In [ ]:
full_run(num_epochs=120, version=-1, force_restart=False)

In [ ]:
full_run(num_epochs=120, version=7, force_restart=False)

In [ ]:
full_run(num_epochs=120, version=0, force_restart=False)

In [ ]:
full_run(num_epochs=120, version=5, force_restart=False)

In [ ]:
full_run(num_epochs=120, version=-3, force_restart=False)